# Sheet 02
This is the implementation of Sheet 02 of Awais Ahmed and Dominic Nieder.

In [461]:
from numpy.typing import NDArray  # trying to be typesafe
import numpy as np  # needed all over the place
from IPython.display import HTML # in line animations

import matplotlib.pyplot as plt  # for plots

from matplotlib import colors  # not quite sure what fore
from matplotlib.ticker import PercentFormatter  # also not sure, maybe animation..?
from matplotlib.animation import FuncAnimation, FFMpegWriter  # for animations

### Particle
* creating "Particle" class, containing all the information of all particle positions, velocities, mass and accellerations as well as interaction potential (all the object of class "Particle" shall interact by a Lennard-Jones Potential)

In [462]:
class Particle:
    def __init__(
        self,
        n: int,
        m: float | int,
        x: NDArray[np.float64],
        y: NDArray[np.float64],
        vx: NDArray[np.float64],
        vy: NDArray[np.float64],
    ):
        self.n: int = n
        self.m: float = float(m)
        self.x: NDArray[np.float64] = np.array(x, dtype=np.float64)
        self.y: NDArray[np.float64] = np.array(y, dtype=np.float64)
        self.vx: NDArray[np.float64] = np.array(vx, dtype=np.float64)
        self.vy: NDArray[np.float64] = np.array(vy, dtype=np.float64)
        self.ax1: NDArray[np.float64] = np.zeros(n, dtype=np.float64)
        self.ay1: NDArray[np.float64] = np.zeros(n, dtype=np.float64)
        self.ax2: NDArray[np.float64] = np.zeros(n, dtype=np.float64)
        self.ay2: NDArray[np.float64] = np.zeros(n, dtype=np.float64)

        
    def __repr__(self):
        return f"This is a particle at {self.x[0]}, {self.y[0]} with v={self.vx[0]},{self.vy[0]} interacting by Lennard-Jones potential."

### Initialisation of particles 
* initialisation on a grid with radom velocity:
* issues pending!

In [463]:
def initialize_particles_on_grid(
    n_particles: int, 
    box: tuple[float|int, float|int], 
    mass: float, 
    velocity: float,
    add_jitter: bool
) -> Particle:
    """
    Particles a regular grid within a given spacing.\n
    Random velocity directions. \n
    Returns: Particle with initialized positions and velocities. 
    """
    x_return = np.zeros(n_particles)
    y_return = np.zeros(n_particles)
    grid_sections = int(np.ceil(np.sqrt(n_particles)))  # find the number of colums & rows

    # even spacing
    x_spacing = box[0]/grid_sections 
    y_spacing = box[1]/grid_sections
    
    # makes grid coordinates
    x_positions, y_positions = np.meshgrid(
        np.arange(grid_sections) * x_spacing, 
        np.arange(grid_sections) * y_spacing
    )
    
    # take n paricles out of the grid
    x_return = x_positions.flatten()[:n_particles]
    y_return = y_positions.flatten()[:n_particles]

    # adding the possibility to a a jitter to the "particle grid" -> What are the effects of jitter to energy init?
    if add_jitter:
        x_positions += (np.random.rand(len(x_positions))-0.5)
        y_positions += (np.random.rand(len(y_positions))-0.5)
        
    # Initialize velocities 
    vx = velocity * (np.random.rand(n_particles) - 0.5) * 2
    vy = velocity * (np.random.rand(n_particles) - 0.5) * 2

    # check initialisation
    # plt.scatter(x_return,y_return)
    # plt.ylabel("x [nm]")
    # plt.xlabel("y [nm]")
    # plt.title("Init positions of particles")
    # plt.savefig("initialisation.png")
    # plt.close()

    # Create and return a Particle
    return Particle(
        n=n_particles,
        m=mass,
        x=x_return,
        y=y_return,
        vx=vx,
        vy=vy
    )

### Forces and Potentials
Lennard-Jones Potential and Force

In [464]:
def lJ_potentil(
        d12: float|NDArray[np.float64]
) -> float|NDArray[np.float64]:
    """
    Lennard Jones Potential
    """
    c_12 = 9.847044*10**(-3)  #  J/mol
    c_6 = 6.2647225           #  J/mol
    return c_12/d12**12-c_6/d12**6

def lj_force(
        d12: float|int,
        r12: NDArray[np.float64]
) -> NDArray[np.float64]:
    """
    Force by Lennard Jones interaction potential.
    """
    c_12 = 9.847044*10**(-3)  # J/mol
    c_6 = 6.2647225           # J/mol
    return (12*c_12/d12**13-6*c_6/d12**7) * r12/d12

### Nearest Neighbour search
* the following functions allow for an appropriat nearest neighbour search

In [465]:
def system_shift(
        x: float,
        box: float
) -> NDArray[np.float64]:
    """
    Calculating shift of one coordinate: S->S' of which p'=box/2
    """
    return x-box/2


def rel_coordiante(
        r1: float,
        r2: NDArray[np.float64],
        box: float|int
) -> NDArray[np.float64]:  
    """ 
    Returns list(aroows):(r1->list(r2))(x|y), for vectorcomponent x or y.
    """
    return (r2-system_shift(r1,box))%box - box/2
    

def distance(
        x: float,
        y: float
) -> float:
    """
    Takes relative ccordinates. \n
    Returns distance between two particles. \n\n
    I somehow have to deal with the possibility of dividing by Zero. My idea is to set distance very large, so that the collision get "ignored".
    """
    
    a= np.sqrt(x*x+y*y)
    if a != 0:
        return a
    else:
        return 100000000

### Velocity Verlet
The next functions are all concerned about the integrations and getting all the accelerations by using the Lennard Jones foce field, that was implemented above.

* integrating over positions

In [466]:
### position update
def integrate_position(
        particles: Particle,
        dt: float|int,
        box: tuple[float|int, float|int]
) -> None:
    """
    Velocity Verlet: Updates positions of particles.
    """
    dx = particles.vx*dt+particles.ax1*dt*dt/2
    dy = particles.vy*dt+particles.ay1*dt*dt/2
    particles.x+=dx 
    particles.y+=dy
    particles.x = particles.x%box[0]
    particles.y = particles.y%box[1]
    pass

* integrating over velocity

In [467]:
### updates velocity vectors
def integrate_velocity(
    particles: Particle,
    dt: float|int
) -> None:
    """
    Velocity Verlet: velocity integration.\n
    Updates velocities of particles.
    """
    particles.vx+=1/2 * (particles.ax1+particles.ax2)*dt
    particles.vy+=1/2 * (particles.ay1+particles.ay2)*dt
    pass

* updating the acceleration

In [468]:
### This funciton sets all the accelerations to zero
def reset_acceleration(
        particles: Particle
) -> None:
    """
    Sets particles.ax2 & particles.ay2 to zeros
    """
    particles.ax2=np.zeros(particles.n, np.float64)
    particles.ay2=np.zeros(particles.n, np.float64)
    pass


### updates acceleration for x+dt additavly, so that the superposition of the force fields get respected
def update_accelerations(
        particles: Particle,
        box: tuple[float|int,float|int],    
) -> None:
    """
    Updates particle.ax2 & particle.ay2 from the particle class.\n
    Issues: creating force variable and relative positions x & y on the fly.
    """
    reset_acceleration(particles)
    for i in range(particles.n):
        # relative coordinates i
        x_rel=rel_coordiante(particles.x[i], particles.x[i+1:particles.n],box[0])  # I don't like to create list on the fly... -> add as field in the class "Particle" or implement functionally?
        y_rel=rel_coordiante(particles.y[i], particles.y[i+1:particles.n],box[1])

        for j in range(i+1,particles.n):
            # calculating the acceleration j->i! 
            acceleration = 1/particles.m * lj_force(
                distance(x_rel[j-(i+1)],y_rel[j-(i+1)]),
                np.array([x_rel[j-(i+1)],y_rel[j-(i+1)]])
                )  # still dont like creating variables on the fly... -> creat force field in class "Particles"...?
            particles.ax2[i] += acceleration[0]  
            particles.ay2[i] += acceleration[1]
            particles.ax2[j] -= acceleration[0] 
            particles.ay2[j] -= acceleration[1]
    pass



### Simulation and main loops
* This function contains the integration funcitons, updates forces 

In [469]:
def iterate(
        dt: float|int,
        particles: Particle,
        box: tuple[float|int,float|int]
    ) -> None:
    """
    Updates positions and velocities. \n
    Accelerations for 1. iteration need to be obtained prior. 
    """
    particles.ax1=particles.ax2  # prior a(t+dt) -> a(t)
    particles.ay1=particles.ay2
    integrate_position(particles,dt,box)
    update_accelerations(particles,box)
    integrate_velocity(particles,dt)
pass

Next the 'main' simulation function is implemented. This iterates over all timesteps and saves the data of interest (phase space).

In [470]:
def simulate(
        particles: Particle, 
        time: int,
        dt: float|int, 
        box: tuple[float|int,float|int],
        data: NDArray[np.float64]
) -> NDArray[np.float64]:
    """
    Runs a simulation of n-particles in a box and saves phase-space coordinates to data.\n
    Returns data.
    """
    update_accelerations(particles,box)  # initial calculation for accelerations
    for t in range(time):  # iteration over all timestepps
        iterate(dt,particles,box)

        ### save data of interest
        data[t,0,:]=particles.x
        data[t,1,:]=particles.y
        data[t,2,:]=particles.vx
        data[t,3,:]=particles.vy
    return data

### Analysis
Here functions that analyse the data will be implemented. Animation, Potential and Kinetic Energies. 
* First of funcitons for animation

In [471]:
def animate(
        frame,
        x_data: NDArray[np.float64],
        y_data, 
        scat
):
    positions = np.c_[x_data[frame, :], y_data[frame,:]]
    scat.set_offsets(positions)
    return scat,

def animation_plot(
    p_radius: float|int,
    box: tuple[float|int]
):
    fig, ax = plt.subplots()
    scat = ax.scatter([], [], s=p_radius)  # Scale size for visibility
    ax.set_xlim(box[0])
    ax.set_ylim(box[1])
    ax.set_title("Simulation Visualization")
    ax.set_xlabel("X [nm]")
    ax.set_ylabel("Y [nm]")
    return scat, fig, ax

def creat_animation(
        save_animation: bool,
        animation_name: str,
        animation_box: tuple[int,int],
        x_data: NDArray[np.float64],
        y_data: NDArray[np.float64]
) -> FuncAnimation: 
    scat, fig, ax = animation_plot(10, animation_box)
    anim = FuncAnimation(fig, animate, frames=len(x_data[:,0]), fargs=(x_data, y_data, scat), interval=20, blit=True)
    writer = FFMpegWriter(fps=10, metadata=dict(artist='Dominic Nieder'), bitrate=1800)
    if save_animation:
        anim.save(animation_name+".mp4")
    return anim



* Analysis of the kinetic Energy: $E=\frac{1}{2} \Sigma_i m_i |v_i|^2$ 

In [472]:
def kinetic_Energy(
        velocity_x_data: NDArray[np.float64],
        velocity_y_data: NDArray[np.float64],
        mass: float | NDArray[np.float64]
) -> NDArray[np.float64]:  
    """
    Takes velocity Data. \n
    Returns kinetic Energy as a funciton of time.
    """
    ekin = mass/2*np.sum((np.square(velocity_x_data)+np.square(velocity_y_data)),1)
    return ekin

In [473]:
a=np.array([[1,1],[2,2],[3,3]])
print(np.square(a))

[[1 1]
 [4 4]
 [9 9]]


* Potential Energy as defined above

In [474]:
def potential_Energy(
    position_x_data: NDArray[np.float64],
    position_y_data: NDArray[np.float64],
    box: tuple[float|int,float|int]
) -> NDArray[np.float64]:
    """
    Takes posiiton arguments x(t), y(t) and a box as input. \n
    Returns Total potential Energy as a function of time.
    """
    e=np.zeros(len(position_x_data[:,0]))
    # iteration over every time step -> function of 
    for T in range(len(position_x_data[:,0])):
        # iteration of particle pairs
        for i in range(len(position_x_data[0,:])):
            dx=rel_coordiante(position_x_data[T,i],position_x_data[T,i+1:len(position_x_data[1])],box[0])
            dy=rel_coordiante(position_y_data[T,i],position_y_data[T,i+1:len(position_y_data[1])],box[1])
            d = map(distance,dx,dy)
            e[T]+= np.sum(np.array(list(map(lJ_potentil,d))))
    return e

* Temperature will be calculated by $<E_{kin}> = \frac{f}{2}kT$ or $T=\frac{2<E_{kin}>}{kf}$, where k is the Boltzmankonstant and Temperature $T$

In [475]:
def temperature_curve(  
        dof: int,  # degrees of freedom
        e_kin: NDArray[np.float64],
        n_particles: int,
) -> NDArray[np.float64]:
    k = 1.380649*10**(-23)
    return 2*(e_kin/n_particles)/dof/k

### Code execution
Next I define all the variables and data list and use the simulation function with particles that shall be initialised on a grid.

In [ ]:
n_particles: int = 64

paritcle_mass: float = 18*10**(-3)  # kg/mol -> same units!

velocity: float = 150 #  nm/ns

time_steps: int = 20000

dt: float = 0.0002  # ns

box: tuple[int, int] = (10,10)

data: NDArray[np.float64] = np.zeros((time_steps,len(box)**2,n_particles)) # (Time, len(x,y,vx,vy), N)

save_mp4: bool = True

data = simulate(
    initialize_particles_on_grid(
    n_particles=n_particles,
    box=box,
    mass=paritcle_mass,  
    velocity=velocity,
    add_jitter=False  
    ),
    time=time_steps,  # timesteps
    dt=dt,  # ns
    box=box,
    data=data  # Time, phase-space pp, particles
)

mp4 = creat_animation(save_mp4, "sheet02-animation3",box, data[:,0,:],data[:,1,:])  
# HTML(mp4.to_html5_video())

* Analysis of Energies

In [ ]:
timestep_array=np.arange(time_steps) * dt
e_kin = kinetic_Energy(data[:,2,:],data[:,3,:],paritcle_mass)
e_pot = potential_Energy(data[:,0,:], data[:,1,:],box)
temperature = temperature_curve(len(box),e_kin, n_particles)

fig, axs = plt.subplots(1, 2, sharey=False, tight_layout=False)

axs[0].plot(timestep_array, e_kin,label="kin.")
axs[0].plot(timestep_array, e_pot,label="pot.")
axs[1].plot(timestep_array, temperature, label="tmep.")
axs[0].set_xlabel('time [ns]')   
axs[1].set_xlabel('time [ns]')
axs[0].set_ylabel('Energy in [J]')  # find correct units of energy
axs[1].set_ylabel('T in [K]') 
axs[0].set_title('Energies')
axs[1].set_title('Temperature')
axs[0].legend()
axs[1].legend()
plt.savefig("sheet02-AnalysisEnergies_and_temp2.png")
plt.close()


